In [1]:
import pickle as pkl
import datasets
import numpy as np
from tqdm.auto import tqdm

In [2]:
from tqdm import tqdm
import pickle
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
    BitsAndBytesConfig,
    LlamaForCausalLM,
    LlamaTokenizer
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model
)
from datasets import Dataset
import evaluate


In [2]:
with open('results_save_influence0.pkl', 'rb') as f:
    IF_dict=pkl.load(f)
print('loaded pickle')
print(IF_dict)
sorted_influences=IF_dict['influence']['proposed'].apply(lambda x: x.argsort(), axis=1)
print(sorted_influences)

loaded pickle
{'runtime': defaultdict(<class 'list'>, {'identity': 2.86102294921875e-06, 'proposed': 76.37844920158386}), 'influence': defaultdict(<class 'list'>, {'identity':           0          1          2          3          4          5   \
0  32.820591  24.366282  20.888435  24.008913  27.876163  19.330626   
1  24.366282  34.807888  16.800751  19.194273  29.856312  14.258027   
2  20.888435  16.800751  27.595709  30.788624  16.551535  24.993111   
3  24.008913  19.194273  30.788624  43.560310  18.507236  31.541586   
4  27.876163  29.856312  16.551535  18.507236  44.374142  13.761898   
5  19.330626  14.258027  24.993111  31.541586  13.761898  30.109024   
6  17.338789  16.158508  16.353416  18.829626  17.886845  15.359276   
7   3.519126   2.729677   4.867253   6.103373   2.321034   5.158979   
8   2.335143   1.782158   3.156395   3.924849   1.622480   3.205610   
9  16.690983  13.632009  21.556187  26.749725  12.114012  21.738220   

          6         7         8          9

In [3]:
train_dataset = datasets.load_from_disk("/nas02/Hadi/Incontenxt-influence/DataInf/datasets/banking77-train-900.hf")
validation_dataset = datasets.load_from_disk("/nas02/Hadi/Incontenxt-influence/DataInf/datasets/banking77-test-100.hf")

In [4]:
all_classes=set(train_dataset['label_text'])

In [5]:
all_classes

{'Refund_not_showing_up',
 'activate_my_card',
 'age_limit',
 'apple_pay_or_google_pay',
 'atm_support',
 'automatic_top_up',
 'balance_not_updated_after_bank_transfer',
 'balance_not_updated_after_cheque_or_cash_deposit',
 'beneficiary_not_allowed',
 'cancel_transfer',
 'card_about_to_expire',
 'card_acceptance',
 'card_arrival',
 'card_delivery_estimate',
 'card_linking',
 'card_not_working',
 'card_payment_fee_charged',
 'card_payment_not_recognised',
 'card_payment_wrong_exchange_rate',
 'card_swallowed',
 'cash_withdrawal_charge',
 'cash_withdrawal_not_recognised',
 'change_pin',
 'compromised_card',
 'contactless_not_working',
 'country_support',
 'declined_card_payment',
 'declined_cash_withdrawal',
 'declined_transfer',
 'direct_debit_payment_not_recognised',
 'disposable_card_limits',
 'edit_personal_details',
 'exchange_charge',
 'exchange_rate',
 'exchange_via_app',
 'extra_charge_on_statement',
 'failed_transfer',
 'fiat_currency_support',
 'get_disposable_virtual_card',
 '

In [6]:
class_id_mapping={}

In [7]:
for k,v in enumerate(train_dataset['label_text']):
    class_id_mapping[k]=v
    

In [8]:
class_id_mapping

{0: 'apple_pay_or_google_pay',
 1: 'apple_pay_or_google_pay',
 2: 'apple_pay_or_google_pay',
 3: 'apple_pay_or_google_pay',
 4: 'apple_pay_or_google_pay',
 5: 'apple_pay_or_google_pay',
 6: 'apple_pay_or_google_pay',
 7: 'apple_pay_or_google_pay',
 8: 'apple_pay_or_google_pay',
 9: 'apple_pay_or_google_pay',
 10: 'apple_pay_or_google_pay',
 11: 'apple_pay_or_google_pay',
 12: 'supported_cards_and_currencies',
 13: 'supported_cards_and_currencies',
 14: 'supported_cards_and_currencies',
 15: 'supported_cards_and_currencies',
 16: 'supported_cards_and_currencies',
 17: 'supported_cards_and_currencies',
 18: 'supported_cards_and_currencies',
 19: 'supported_cards_and_currencies',
 20: 'supported_cards_and_currencies',
 21: 'supported_cards_and_currencies',
 22: 'supported_cards_and_currencies',
 23: 'supported_cards_and_currencies',
 24: 'pending_cash_withdrawal',
 25: 'pending_cash_withdrawal',
 26: 'pending_cash_withdrawal',
 27: 'pending_cash_withdrawal',
 28: 'pending_cash_withdrawal'

In [9]:
with open('../results_save_influencebanking.pkl', 'rb') as f:
    IF_dict=pkl.load(f)
# print('loaded pickle')
# print(IF_dict)
sorted_influences=IF_dict['influence']['proposed'].apply(lambda x: x.argsort(), axis=1)
print(sorted_influences)

     0    1    2    3    4    5    6    7    8    9    ...  914  915  916  \
0    598   22  468  121  243  477  251  515  297  256  ...    0    6  512   
1    598  121  505   22  251  477  662  256  515  486  ...  677  350   34   
2    121  481  598  256  486  251  487  855  170  583  ...  101   18  878   
3     22  598  505  256  515  243   48  481  468  121  ...    5  813  575   
4    598  121   22  243  505  256  251  248  662   63  ...  119   33  116   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
149   22  515  341  343  121  459  488  632  505  490  ...  391  278  843   
150   22  256  505  598  243  515  297  121  468  477  ...   51  866  268   
151  505   22  515  336  468  121  297   48  632  256  ...  691  612  903   
152  256  505   22  598  243  297  481  336  121   48  ...  440   34  144   
153  598  256  505   22  297  243  336  855  121  481  ...  148  137  210   

     917  918  919  920  921  922  923  
0    813   34    2  814  715  677 

In [10]:
sorted_influences.iloc[0][::-1]

923      5
922    677
921    715
920    814
919      2
      ... 
4      243
3      121
2      468
1       22
0      598
Name: 0, Length: 924, dtype: int64

In [17]:
new_classes=[]
new_class_names=[]
query_iter=np.arange(154)
for idx in query_iter:
    #print(len(sorted_influences.iloc[idx]))
    classes_picked=[]
    ids_picked=[]
    ids=sorted_influences.iloc[idx][::-1]
    # print(ids)
    for id in ids:
        if len(ids_picked)==8:
            new_class_names.append(classes_picked)
            break
        elif id not in ids_picked and class_id_mapping[id] not in classes_picked: #pick top most id and check its class
            ids_picked.append(id)
            classes_picked.append(class_id_mapping[id])
        elif set(classes_picked)==set(all_classes):
            classes_picked=[]
            if id not in ids_picked and class_id_mapping[id] not in classes_picked: #pick top most id and check its class
                ids_picked.append(id)
                classes_picked.append(class_id_mapping[id])
        else:
            continue
    new_classes.append(ids_picked)

In [15]:
classes=[]
old_classes=[]
for idx in query_iter:
    temp=[]
    #print(len(sorted_influences.iloc[idx]))
    
    ids=sorted_influences.iloc[idx][::-1][0:8]
    old_classes.append(ids)
    for id in ids:
        temp.append(class_id_mapping[id])
    
    classes.append(temp)
        

In [16]:
classes[9]

['exchange_via_app',
 'receiving_money',
 'exchange_via_app',
 'receiving_money',
 'receiving_money',
 'receiving_money',
 'exchange_rate',
 'receiving_money']

In [13]:
old_classes[9]

923    560
922     55
921    554
920     57
919     51
918     54
917    469
916     49
Name: 9, dtype: int64

In [18]:
new_class_names[9]

['exchange_via_app',
 'receiving_money',
 'exchange_rate',
 'getting_virtual_card',
 'fiat_currency_support',
 'top_up_by_cash_or_cheque',
 'exchange_charge',
 'getting_spare_card']

In [14]:
new_classes[9]

[560, 55, 469, 638, 101, 885, 702, 179]

In [3]:
class LORAEngineGeneration(object):
    def __init__(self, 
                base_path,
                project_path,
                dataset_name='math_with_reason',
                device="cuda"):
        self.base_path = base_path
        self.project_path = project_path
        #self.adapter_path = "/nas02/Hadi/Incontenxt-influence/DataInf/llama2-70b-gsm8k-justlora" #changed for LESS
        #self.dataset_name = dataset_name
        self.device=device
        self.load_pretrained_network()
        self.load_datasets()   #need to change

    def load_pretrained_network(self):
        # setup tokenizer
        self.tokenizer = LlamaTokenizer.from_pretrained(self.base_path)
        self.tokenizer.padding_side = "right"
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        # load a base model
        quantization_config = BitsAndBytesConfig(load_in_8bit=True, load_in_4bit=False)
        base_model = LlamaForCausalLM.from_pretrained(
            self.base_path,
            quantization_config=quantization_config,
            torch_dtype=torch.bfloat16,
            offload_folder="offload",
            offload_state_dict=True,
            device_map='auto'
        )

        # load a pre-trained model.
        # self.model = PeftModel.from_pretrained(base_model, self.adapter_path, is_trainable=True)
        # self.finetuned_config = LoraConfig.from_pretrained(pretrained_model_name_or_path=self.adapter_path)  #Changed for LESS
        self.model=base_model

    def load_datasets(self):
        # print('qnli')
        self.train_dataset = Dataset.load_from_disk("/nas02/Hadi/Incontenxt-influence/DataInf/datasets/gsm8k-train-900.hf")
        self.validation_dataset = Dataset.load_from_disk("/nas02/Hadi/Incontenxt-influence/DataInf/datasets/gsm8k-test-100.hf")

    def create_tokenized_datasets(self):
        tokenize_func = lambda x: self.tokenizer(
            x["prompt"], truncation=True, padding=True, max_length=128, return_tensors="pt" # text should be more appropritate
        ).to(self.device)

        # if 'with_reason' in self.dataset_name:
        #     column_list=["text", "answer", "variation", "prompt", "reason"]
        # else:
        #     column_list=["text", "answer", "variation", "prompt"]
        
        # column_list_train=['question', 'sentence', 'label', 'idx', 'text', 'prompt'] # QNLI
        # column_list_test=['question', 'sentence', 'label', 'idx', 'text', 'prompt']
        
        # column_list_train=['article', 'label', 'text',"prompt"]  #AGNEWS
        # column_list_test=['article', 'label', 'text',"prompt"]
        
        # column_list_train=['question', 'answer', 'text']
        # column_list_test=['question', 'answer', 'text']
        
        # column_list_train=['premise', 'hypothesis', 'label', 'idx', 'text', 'prompt']  #MNLI
        # column_list_test=['premise', 'hypothesis', 'label', 'idx', 'text', 'prompt']
        
                
        column_list_train=['question', 'answer', 'text', 'prompt']  #GSM8k
        column_list_test=['question', 'answer', 'text', 'prompt']
        
        # column_list_train=['sentence', 'label', 'idx', 'text', 'prompt']  #SST2
        # column_list_test=['sentence', 'label', 'idx', 'text', 'prompt'] 
        
        
        # column_list_train=['question1', 'question2', 'label', 'idx', 'text', 'prompt']  #QQP
        # column_list_test=['question1', 'question2', 'label', 'idx', 'text', 'prompt']
        
        # column_list_train=['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label', 'context', 'text', 'prompt'] #hellaswag
        # column_list_test=['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label', 'context', 'text', 'prompt']

        # column_list_train=['id', 'question', 'question_concept', 'choices', 'answerKey', 'context', 'text', 'prompt']  #cmsqa
        # column_list_test=['id', 'question', 'question_concept', 'choices', 'answerKey', 'context', 'text', 'prompt']
        
        # column_list_train=['question', 'subject', 'choices', 'answer', 'context', 'text', 'prompt']  #mmlu
        # column_list_test=['question', 'subject', 'choices', 'answer', 'context', 'text', 'prompt']
        
        # column_list_train=['text_old', 'label', 'label_text', 'context', 'text', 'prompt']  #banking77
        # column_list_test=['text_old', 'label', 'label_text', 'context', 'text', 'prompt']
        
        # column_list_train=['original_text', 'label', 'context', 'text', 'prompt', 'label_text'] #sarcasm
        # column_list_test=['original_text', 'label', 'context', 'text', 'prompt', 'label_text']
        
        # column_list_train=['original_text', 'label', 'context', 'text', 'prompt', 'label_text'] #tweetevalirony
        # column_list_test=['original_text', 'label', 'context', 'text', 'prompt', 'label_text']
        
        tokenized_datasets=dict()
        tokenized_datasets["train"] = self.train_dataset.map(
            tokenize_func,
            batched=True,
            remove_columns=column_list_train,
        )
        tokenized_datasets["validation"] = self.validation_dataset.map(
            tokenize_func,
            batched=True,
            remove_columns=column_list_test,
        )
        collate_fn = lambda x: self.tokenizer.pad(x, padding="longest", return_tensors="pt")
        

        
        return tokenized_datasets, collate_fn

    
    def compute_gradient(self, tokenized_datasets, collate_fn):    #LESS
        train_dataloader_stochastic = DataLoader(tokenized_datasets["train"], 
                                                  shuffle=False,
                                                  collate_fn=collate_fn,
                                                  batch_size=1)
        val_dataloader_stochastic = DataLoader(tokenized_datasets["validation"], 
                                                  shuffle=False,
                                                  collate_fn=collate_fn,
                                                  batch_size=1)
        print(type(train_dataloader_stochastic))
        print(train_dataloader_stochastic)
        # Compute the gradient
        self.model.eval()
        tr_grad_dict = {}
        for step, batch in enumerate(tqdm(train_dataloader_stochastic)):
            # if step==10:
            #     break
            self.model.zero_grad() # zeroing out gradient

            print(batch.keys())
            
            batch['labels'] = batch['input_ids']
            batch.to(self.device)
            loss = self.model(**batch).loss
            loss.backward()
            
            grad_dict={}
            # break
            # vectorized_grads = torch.cat(
            #     [p.grad.view(-1) for k,p in self.model.named_parameters() if p.grad is not None])
            for k, v in self.model.named_parameters():
                # print(k)
                # print("Shape of layer {} is {}".format(k,v.shape))
                if v.grad is not None: # and 'layernorm' in k and (".0." in k or ".1." in k or "78" in k or "79" in k):
                    # print(v.grad.view(-1).cpu().shape)
                    # print(v.grad.view(-1).cpu())
                    print(k)
                    print(v.shape)
                    # grad_dict[k]=v.grad.view(-1).cpu()
  
            break
                    
            tr_grad_dict[step]=grad_dict
            
            del grad_dict
            
            
        # val_grad_dict = {}
        # for step, batch in enumerate(tqdm(val_dataloader_stochastic)):
        #     # if step==10:
        #     #     break
        #     self.model.zero_grad() # zeroing out gradient
 
        #     batch['labels'] = batch['input_ids']
        #     batch.to(self.device)
        #     loss = self.model(**batch).loss
        #     loss.backward()
            
        #     grad_dict={}
        #     # vectorized_grads = torch.cat(
        #     #     [p.grad.view(-1) for k,p in self.model.named_parameters() if p.grad is not None])
        #     for k, v in self.model.named_parameters():
        #         if v.grad is not None and 'layernorm' in k and (".0." in k or ".1." in k or "78" in k or "79" in k):
        #             grad_dict[k]=v.grad.view(-1).cpu()
                    
        #     val_grad_dict[step]=grad_dict
            
        return tr_grad_dict



In [4]:
!nvidia-smi

Sun May 12 01:40:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off | 00000000:03:00.0 Off |                  Off |
| 30%   31C    P8              22W / 300W |  41039MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5,6"

In [7]:
base_path = "/nas02/Hadi/Incontenxt-influence/DataInf/llama-2-13b-chat-converted" 
project_path ="/nas02/Hadi/Incontenxt-influence/DataInf" 
lora_engine = LORAEngineGeneration(base_path=base_path, 
                                project_path=project_path,
                                dataset_name='math_with_reason')        

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5,6"
tokenized_datasets, collate_fn = lora_engine.create_tokenized_datasets()
tr_grad_dict, val_grad_dict = lora_engine.compute_gradient(tokenized_datasets, collate_fn)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Parameter 'function'=<function LORAEngineGeneration.create_tokenized_datasets.<locals>.<lambda> at 0x7f5800d72ca0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<class 'torch.utils.data.dataloader.DataLoader'>


  0%|          | 0/900 [00:00<?, ?it/s]

dict_keys(['input_ids', 'attention_mask'])


/home/haskari/miniconda3/envs/influence/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
  0%|          | 0/900 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 7 has a total capacity of 47.51 GiB of which 1.56 MiB is free. Process 1063630 has 44.25 GiB memory in use. Including non-PyTorch memory, this process has 3.24 GiB memory in use. Of the allocated memory 2.27 GiB is allocated by PyTorch, and 480.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 